##### This file describes how Black derman toy model of interst rate is obtained and then subsequently used to price options on bonds


The Black Derman Toy model is used to price bonds and interest rate models. It is a one factor model and has the following dynamics:  
$d\ln(r)= \{\theta(t)+\dot{\sigma}/{\sigma}\ln(r)\}dt+\sigma(t)dW$

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
def BlackDermanToyInterestRate(ZCRates,ZCVol,ParVal):
    # this function computes the short rate for BDT model
    # MatPeriod is years to maturity of Zero coupon Bond
    # ZCRate is the zero coupon rate in %
    # ZCVol is the zero coupon Volatility in %
    # FVal face value of ZC-Bond
    
    treelength = len(ZCRates); # length of the binomial tree
    
    ZCBPrice =  np.zeros(treelength+1);# array for Zero coupon bond price
    
    # computing price of Zero Coupon Bond
    for i in range(treelength+1):
        ZCBPrice[i] = ParVal/(1+ZCRates[i-1])**i;
        
    SRate = np.zeros([treelength,treelength]);# matrix of short rate 
   
    ZCBPrice = ZCBPrice[1:];
      
    SRate[0,0] = ZCRates[0];
     
    for i in range(1,treelength):
        
        guess = 0.0787
        bsum = 0
        
    
        while bsum!=ZCBPrice[i]:
            
            Sigma = np.exp(2*ZCVol[i]); 
            bsum = 0;
            
            if  i==1:
                Dfact =1;
            elif i==2:
                Dfact = np.prod(SRate[0,:]+1);
            else:
                for k in range(i-2):
                    Dfact = np.prod(SRate[k,:]+1);
            
            
            for j in range(treelength):
                if j<i:
                    bsum = bsum +(0.5**i*ParVal/(1+guess*Sigma**j)+0.5**i*ParVal/(1+guess*Sigma**(j+1)))/(Dfact*(1+SRate[i-1,j]))    
                  
                else:
                    break;
                    
              # guessing for bisection method check this part     
            if (bsum-ZCBPrice[i])<0.01:
                SRate[i,j] = guess;
                break;
            elif bsum<ZCBPrice[i]:
                guess = guess+2*(1/bsum-1/ZCBPrice[i]);
            elif bsum>ZCBPrice[i]:
                guess = guess-2*(1/ZCBPrice[i]-1/bsum);
           
        length = j+1;
        Temp = np.zeros(length); 
        m = j;
        
        for k in range(length):            
            Temp[k] = SRate[i,j]*Sigma**m;
            m = m-1
        
        Temp = np.flip(Temp);
        
        for j in range(treelength):
            if j<i+1:
                SRate[i,j] = Temp[j];
            else:
                break;
                
    
    return SRate;      


In [9]:
ZCRates = np.array( [9,9.5,10,10.5,11]);
ZCRates = ZCRates/100;
ZCVol = np.array([24,22,20,18,16]);
ZCVol = ZCVol/100;
ParVal = 100
SRate = BlackDermanToyInterestRate(ZCRates,ZCVol,ParVal);

In [10]:
print (SRate)

[[0.09       0.         0.         0.         0.        ]
 [0.0787     0.12219806 0.         0.         0.        ]
 [0.0787     0.1174066  0.17515007 0.         0.        ]
 [0.0787     0.11280302 0.16168389 0.23174628 0.        ]
 [0.0787     0.10837996 0.14925305 0.20554051 0.28305555]]


In [ ]:
def BondPriceStructure(SRate,ParVal):
    # this function computes the bond price strutrure for the given short rate tree
    
    (d1,d2) = np.size(SRate);
    
    BPrice = np.zeros([d1,d2]);
    BPrice[:,d2] = ParVal
    
    for i in range(d2,-1,0):
        for j in range(d1):
            BPrice[i-1,j] = BPrice[i,j]/(1+SRate[i,j]);
   
    return BPrice

In [ ]:
def OptionPricingBDT(BPrice,SRate,StrikePrice,Callablity):
    # this function computes the price of the embeded option on the bond
    
    (d1,d2) = np.size(BPRice);
    OptionPayoff = Max(BPrice-StrikePrice,0);
    
    OptionPrice = np.zeros([d1,d2]);
    
    for i in range(d1,-1,0):
        for j in range(d2):
            OPtionPrice[:,j] = (0.5*OptionPayoff[i+1,j]+0.5*OptionPayoff[i,j])/(1+SRate[i-1,j]) 
            
    return OptionPrice

In [ ]:
# The following lines of code compute the required OAD
def CallableBondPrice(ZCRates,ZCVol,ParVal,StrikePrice,Callabality):
    
    SRate = BlackDermanToyInterestRate(ZCRates,ZCVol,ParVal);
    
    BPrice = BondPriceStructure(SRate,ParVal);
    
    OptionPrice = OptionPricingBDT(BPrice,SRate,StrikePrice,Callablity);
    
    CBPrice = BPrice - OptionPrice;
    
    return CBPRice;

In [ ]:
# define paramateres

# get the callabale bond price

# Bond price with the interest rate stressed Up

# Bond price with the interest rate stressed Down

# computing OAD



